# ТЕСТОВОЕ ЗАДАНИЕ от Комолина Артёма

## Установка графовых баз данных

Для начала установим Neo4j и ArangoDB
### Neo4j
Скачиваем с официального сайта

![photo](photos/1.jpg)

После установки и запуска программы, создаем проект и базу данных

![photo](photos/2.jpg)


### ArangoDB 

Для установки достаточно воспользоваться Docker и прописать команду в командной строке:

![photo](photos/3.jpg)

NO_AUTH значит, что мы заходим без авторизации
8529 - порт
--name test-data-arango arangodb - значит открываем проект под именем test-data-arango в образе arangodb

Заходим на localhost:8529

![photo](photos/4.jpg)

### Далее будем работать с Neo4j

Но для начала нам нужно подготовить наш csv файл

## Подготовка CSV файла

В Neo4j мы будем использовать язык запросов <b>Cypher</b>

Чтобы cypher правильно обращался к нашему csv файлу, в качестве разделителя будем использовать запятую.

1) Поменяем все наши ';' на ','
2) Заголовки сделаем без пробелов, это нам поможет делать cypher запросы без оошибок

In [1]:
import csv

#Прочитаем наш csv файл и создадим список data из словарей, в качестве ключа заголовки csv файла
with open("data_test.csv", "r", encoding="utf_8_sig") as f:
    reader = csv.reader(f, delimiter=";")

    data = []
    for row in reader:
        data.append({'id_события': row[0], 'ФИО_участника_события_1': row[1], 'ФИО_участника_события_2': row[2]})

#Удалим повторяющиеся заголовки
del data[0]

#запишем новый файл data.csv
with open("data.csv", "w", encoding="utf_8_sig") as f:
    writer = csv.DictWriter(f, fieldnames=list(data[0].keys()),  delimiter=',')
    writer.writeheader()
    for d in data:
        writer.writerow(d)

У нас сохранится новый файл data.csv в том же каталоге.

Выглядит он вот так:

![photo](photos/5.jpg)

Точки с запятой поменялись на запятые. Значит все хорошо! Идем дальше!


## Создаем графовую базу данных в Neo4j на основе data.csv

Положим наш новенький data.csv в папку import нашего проека в (C:\Users\User\.Neo4jDesktop\relate-data\dbmss\dbms-ecdc0fa0-514d-4151-afb3-ac2364c9b5e1\import)

Запустим и откроем нашу созданную базу данных в Neo4j 

![photo](photos/6.jpg)

Выполним cypher запрос:

![photo](photos/15.jpg)

Наша графовая база создаласть:

![photo](photos/9.jpg)
![photo](photos/10.jpg)

## Найдем взаимосвязи

### Визуально

![photo](photos/11.jpg)

### Алгоритмизируем

Найдем такие сложные взаимосвязи, например, возможно один и тот же человек участвовал в нескольких событиях.

Найдем этих людей с помощью такого Cypher запроса:

![photo](photos/12.jpg)
### Мы нашли людей из ФИО участника события 1, которые участвовали в нескольких событиях. И вывели списком friends имена тех людей с которыми они взаимодейсвовали

### Теперь найдем людей из ФИО участника события 2, участвовавших в нескольких событиях

![photo](photos/13.jpg)

### Например, покажем связи участника "Ахромеева Алина Ивановна"

![photo](photos/14.jpg)

### Анализируем

Анализируя взаимосвязи участников, можно сделать выводы:
1) В большинстве люди участвуют в событии единожды 
2) Есть люди которые взаимодействуют больше одного раза
3) Люди, которые участвуют в событиях больше одного раза скорее всего не связаны друг с другом, и их связи случайные. Иногда встречаются повторые взаимодействия одних и тех же людей, но это редко. Например: Двигубская -> Шолохов, Шолохов -> Двигубская.

### Можно предположить, что это данные транзакций (в которых не указана сумма перевода, дата, а только id транзакции). Один человек отправляет сумму другому и id события записывается. Возможно люди, которые часто учавствуют в событиях, например Ахромеева Алина Ивановна, являются трейдерами, торгуют валютой и зарабатывают на этом.



## Напишем сервис, который создает JSON файл

Пример поиска по имени 'Ахромеева Алина Ивановна'

In [2]:
import json
from neo4j import GraphDatabase

#подключаемся к базе Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("data_test", "data1234"))


#Считываем данные в нужном формате
def get_related_objects(name):
    with driver.session() as session:
        result = session.run(f"MATCH (n)-[r]->(m) WHERE n.name = {name} RETURN n, r, m", name=name)
        objects = []
        for row in result:
            objects.append({'id события': row["r"]["id"], 'ФИО участника события 1': row["n"]["name"], 'ФИО участника события 2': row["m"]["name"]})
        return objects

name = get_related_objects("'Ахромеева Алина Ивановна'")

#Создаем json файл
with open("data_j.json", "w", encoding="utf_8_sig") as f:
    json.dump(name, f, indent=4, ensure_ascii=False)

# Наш JSON файл

In [ ]:
[
    {
        "id события": "42389",
        "ФИО участника события 1": "Ахромеева Алина Ивановна",
        "ФИО участника события 2": "Ящукова Любовь Ефимовна"
    },
    {
        "id события": "53707",
        "ФИО участника события 1": "Ахромеева Алина Ивановна",
        "ФИО участника события 2": "Тяжлов Ринат Владиславович"
    },
    {
        "id события": "59801",
        "ФИО участника события 1": "Ахромеева Алина Ивановна",
        "ФИО участника события 2": "Соломеина Кристина Георгиевна"
    },
    {
        "id события": "61824",
        "ФИО участника события 1": "Ахромеева Алина Ивановна",
        "ФИО участника события 2": "Дудыкина Мария Романовна"
    },
    {
        "id события": "89429",
        "ФИО участника события 1": "Ахромеева Алина Ивановна",
        "ФИО участника события 2": "Щенников Дмитрий Григорьевич"
    },
    {
        "id события": "90615",
        "ФИО участника события 1": "Ахромеева Алина Ивановна",
        "ФИО участника события 2": "Камчадалов Артем Ярославович"
    },
 .
 .
 .
]